In [1]:
import json
from fuzzywuzzy import fuzz
import psycopg2
import pyperclip
import tkinter as tk
from tkinter import messagebox
import sys

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [11]:
# Чтение json конфига
with open('config.json') as file:
    config = json.load(file)

pgre_login = config['postgre login']
pgre_password = config['postgre password']
pgre_host = config['postgre host']
pgre_port = config['postgre port']
pgre_db = config['postgre database']

#Подключение к postgres
conn = psycopg2.connect(
    host = pgre_host,
    port = pgre_port,
    database = pgre_db,
    user = pgre_login,
    password = pgre_password
)

In [12]:
dupl_id_list = (116711493)

In [26]:
dulp_cycle_cursor = conn.cursor()
query = """
select id, duplicate_id
from av_full
where duplicate_id = %s
""" % (dupl_id_list)
dulp_cycle_cursor.execute(query)
d_c_r = dulp_cycle_cursor.fetchall()

In [27]:
id_to_double_check = []
for i in d_c_r:
   id_to_double_check.append(i[0])

In [28]:
id_to_double_check

[Decimal('114108454'),
 Decimal('107392622'),
 Decimal('108416353'),
 Decimal('109113459'),
 Decimal('109528932'),
 Decimal('114426235'),
 Decimal('112723332'),
 Decimal('110432351'),
 Decimal('114579056'),
 Decimal('115734647'),
 Decimal('110173767'),
 Decimal('111101772'),
 Decimal('113829991'),
 Decimal('113934875'),
 Decimal('109331244')]